In [1]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import re

In [2]:
def show_files(folder):
    for ind in range(len(folder)):
        print(f"{ind:3}: {folder[ind]}")

In [3]:
img_folder = "E:/Personal/Albums/"
arr = os.listdir(img_folder)
arr.sort()
arr

['Arqui',
 'Caderno',
 'Canada',
 'Cancun',
 'Creme',
 'Eu',
 'Faaps',
 'Fabs',
 'Fenix',
 'Hapkido',
 'Itapeva',
 'Lara',
 'LeWagon',
 'Mackenzie',
 'Memes',
 'Miau',
 'Parkour',
 'Perfil',
 'Random',
 'Riviera',
 'Santa',
 'Tech']

In [4]:
jpgs = glob.glob(img_folder + '/**/*.jpg', recursive=True)
pngs = glob.glob(img_folder + '/**/*.png', recursive=True)
jpegs = glob.glob(img_folder + '/**/*.jpeg', recursive=True)
webps = glob.glob(img_folder + '/**/*.webp', recursive=True)

images = jpgs + jpegs + pngs + webps
print(len(images))

1663


In [5]:
img_paths = []
for ind in range(len(arr)):
    path = f"{img_folder}{arr[ind]}"
    img_paths.append(path)

show_files(img_paths)

  0: E:/Personal/Albums/Arqui
  1: E:/Personal/Albums/Caderno
  2: E:/Personal/Albums/Canada
  3: E:/Personal/Albums/Cancun
  4: E:/Personal/Albums/Creme
  5: E:/Personal/Albums/Eu
  6: E:/Personal/Albums/Faaps
  7: E:/Personal/Albums/Fabs
  8: E:/Personal/Albums/Fenix
  9: E:/Personal/Albums/Hapkido
 10: E:/Personal/Albums/Itapeva
 11: E:/Personal/Albums/Lara
 12: E:/Personal/Albums/LeWagon
 13: E:/Personal/Albums/Mackenzie
 14: E:/Personal/Albums/Memes
 15: E:/Personal/Albums/Miau
 16: E:/Personal/Albums/Parkour
 17: E:/Personal/Albums/Perfil
 18: E:/Personal/Albums/Random
 19: E:/Personal/Albums/Riviera
 20: E:/Personal/Albums/Santa
 21: E:/Personal/Albums/Tech


In [6]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)
model.cuda()

Using cache found in C:\Users\Ricardo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-6 Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 3072MiB)

Fusing layers... 
YOLOv5l6 summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1

In [7]:
DG = nx.DiGraph()

In [8]:
for img in images:
    ind = images.index(img)
    img_name = img[len(img_folder):].replace("\\", "/")
    print(f"({ind + 1:4d}/1663) => {img}", end=" ")

    result = model(img, size=1024) # múltiplo de 32
    print('processed! with tags:', end='')

    tags = result.pandas().xyxy[0][['name']].groupby("name").value_counts()

    for j in range(len(tags)):
        tag_name = tags.index[j]
        tag_quant = tags.iloc[j]
        print(f' {tag_name}', end='')
        
        DG.add_edge(tag_name, img_name) # ligando nome da classe identificada com o nome do arquivo
        if tag_name == 'person':
            if tag_quant == 2:
                DG.add_edge('pair', img_name)
            elif tag_quant == 3:
                DG.add_edge('trio', img_name)
            if tag_quant > 3:
                DG.add_edge('group', img_name)
        if tag_name == 'cat' or tag_name == 'dog':
            DG.add_edge('pets', img_name)

    if len(tags) == 0:
        DG.add_node(img_name)
    
    print('')


# results = model(images, size=512) # múltiplo de 32

(   1/1663) => E:/Personal/Albums\Arqui\IMG-20201019-WA0030.jpg processed! with tags: person
(   2/1663) => E:/Personal/Albums\Arqui\IMG-20180718-WA0004.jpg processed! with tags: cup dining table dog person teddy bear
(   3/1663) => E:/Personal/Albums\Arqui\IMG-20200928-WA0029.jpg processed! with tags: chair person
(   4/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0001.jpg processed! with tags: cup person
(   5/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0102.jpg processed! with tags: person
(   6/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0104.jpg processed! with tags: person
(   7/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0106.jpg processed! with tags: person
(   8/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0115.jpg processed! with tags: car person
(   9/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0116.jpg processed! with tags: person
(  10/1663) => E:/Personal/Albums\Arqui\IMG-20201011-WA0117.jpg processed! with tags: person
(  11/1663) => E:/Person

In [9]:
print(DG.number_of_nodes() - len(images))
print(DG.number_of_edges())
pattern = re.compile(r'.+[.].+')
tags_res = set(DG.nodes()).copy()
tags_res -= {item for item in DG.nodes if pattern.match(item)}

84
4495


In [14]:
nx.draw_shell(DG, with_labels=False, font_weight='bold')

plt.savefig("results/path.png")
plt.clf()

The file results/path.png already exists. overwriting...


In [17]:
pos = nx.bipartite_layout(DG, tags_res)
nx.draw(DG, pos, with_labels=False, font_weight='bold')

plt.savefig('results/columns_old.png')
plt.clf()

In [19]:
tag_options = {
    "node_color" : "red",
    "node_size" : 75,
    "font_size" : 12,
    "font_color" : "black",
    "horizontalalignment" : "right",
    "verticalalignment" : "bottom",
}

photo_options = {
    "node_color" : "blue",
    "node_size" : 50,
    "font_size" : 12,
    "font_color" : "black",
    "horizontalalignment" : "left",
    "verticalalignment" : "center",
}

edge_options = {
    "edge_color" : "black",
    "width" : 1.5,
    "alpha" : .75
}

# plt.figure(figsize=(8, 6))
# plt.axis([-0.65, 0.65, -1, 1])
plt.box(False)
pos = nx.bipartite_layout(DG, tags_res)
for node_pos in pos.values():
  node_pos[0] *= 0.4
  print(node_pos)


nx.draw_networkx_edges(DG, pos, **edge_options)

# TAGS
tags_dict = {n:n for n in DG if n in tags_res}
nx.draw_networkx(DG, pos, nodelist=tags_res, with_labels=True, labels=tags_dict, **tag_options)

# PHOTOS
photos_dict = {n:n for n in DG if n not in tags_res}
nx.draw_networkx(DG, pos, nodelist=list(set(DG.nodes) - set(tags_res)), with_labels=True, labels=photos_dict, **photo_options)


plt.show()
plt.savefig(f"results/columns.png")
plt.clf()

[       -0.4    -0.39394]
[       -0.4    -0.38445]
[       -0.4    -0.37496]
[       -0.4    -0.36546]
[       -0.4    -0.35597]
[       -0.4    -0.34648]
[       -0.4    -0.33699]
[       -0.4    -0.32749]
[       -0.4      -0.318]
[       -0.4    -0.30851]
[       -0.4    -0.29902]
[       -0.4    -0.28952]
[       -0.4    -0.28003]
[       -0.4    -0.27054]
[       -0.4    -0.26105]
[       -0.4    -0.25155]
[       -0.4    -0.24206]
[       -0.4    -0.23257]
[       -0.4    -0.22308]
[       -0.4    -0.21358]
[       -0.4    -0.20409]
[       -0.4     -0.1946]
[       -0.4    -0.18511]
[       -0.4    -0.17561]
[       -0.4    -0.16612]
[       -0.4    -0.15663]
[       -0.4    -0.14713]
[       -0.4    -0.13764]
[       -0.4    -0.12815]
[       -0.4    -0.11866]
[       -0.4    -0.10916]
[       -0.4   -0.099672]
[       -0.4   -0.090179]
[       -0.4   -0.080687]
[       -0.4   -0.071194]
[       -0.4   -0.061702]
[       -0.4   -0.052209]
[       -0.4   -0.042717]
[       -0.4

In [23]:
for tag_name in tags_res:

    # Create a subgraph containing the central node and its neighbors
    subgraph_nodes = [tag_name] + list(DG.neighbors(tag_name))
    subgraph = DG.subgraph(subgraph_nodes)

    # Draw the subgraph
    pos = nx.spring_layout(subgraph)  # You can choose a different layout if needed
    nx.draw(subgraph, pos, with_labels=True, node_color='skyblue', node_size=50, font_size=6, font_color='black')
    labels = {node: node for node in subgraph.nodes()}  # Labels for nodes
    nx.draw_networkx_labels(subgraph, pos, labels)

    # Customize the appearance of the plot
    plt.title("Subgraph with Central Node")
    plt.axis('off')  # Turn off the axis
    plt.show()
    plt.savefig(f"results/spring_{tag_name}.png")
    print(f"created: results/spring_{tag_name}.png!")
    plt.clf()

created: results/spring_bird.png!
created: results/spring_tennis racket.png!
created: results/spring_truck.png!
created: results/spring_bottle.png!
created: results/spring_giraffe.png!
created: results/spring_carrot.png!
created: results/spring_bowl.png!
created: results/spring_fork.png!
created: results/spring_teddy bear.png!
created: results/spring_snowboard.png!
created: results/spring_baseball bat.png!
created: results/spring_trio.png!
created: results/spring_car.png!
created: results/spring_sandwich.png!
created: results/spring_stop sign.png!
created: results/spring_backpack.png!
created: results/spring_hot dog.png!
created: results/spring_boat.png!
created: results/spring_book.png!
created: results/spring_hair drier.png!
created: results/spring_frisbee.png!
created: results/spring_bed.png!
created: results/spring_cow.png!
created: results/spring_pets.png!
created: results/spring_suitcase.png!
created: results/spring_tv.png!
created: results/spring_motorcycle.png!
created: results